### 堆叠受限玻尔兹曼机（RBM）原理与代码实现总结
---
#### **一、RBM原理概述**
受限玻尔兹曼机（Restricted Boltzmann Machine）是一种基于能量的概率图模型，由可见层（Visible Layer）和隐层（Hidden Layer）组成，层内无连接，层间全连接。其核心是通过无监督学习学习数据的潜在特征分布。
##### **1. 模型结构**
- **可见层（v）**：输入数据的显式表示（如像素值）。
- **隐层（h）**：提取的潜在特征。
- **权重矩阵（W）**：连接可见层与隐层的权重。
- **偏置**：可见层偏置（b）和隐层偏置（c）。
##### **2. 能量函数与概率分布**
RBM的能量函数定义为：
$$
E(\mathbf{v}, \mathbf{h}) = -\mathbf{v}^T W \mathbf{h} - \mathbf{b}^T \mathbf{v} - \mathbf{c}^T \mathbf{h}
$$
联合概率分布通过玻尔兹曼分布给出：
$$
P(\mathbf{v}, \mathbf{h}) = \frac{ e^{-E(\mathbf{v}, \mathbf{h})} }{Z}
$$
其中 $ Z $ 为配分函数（归一化因子）。可见层的边缘分布为：
$$
P(\mathbf{v}) = \sum_{\mathbf{h}} P(\mathbf{v}, \mathbf{h})
$$
##### **3. 条件独立性**
由于层内无连接，给定可见层时隐层条件独立，反之亦然：
$$
P(h_j=1|\mathbf{v}) = \sigma\left(\sum_i W_{ij} v_i + c_j\right)
$$
$$
P(v_i=1|\mathbf{h}) = \sigma\left(\sum_j W_{ij} h_j + b_i\right)
$$
其中 $\sigma(x) = \frac{1}{1+e^{-x}}$ 为Sigmoid激活函数。
##### **4. 训练目标**
通过最大化似然函数学习参数（W, b, c）。目标函数为负对数似然：
$$
\mathcal{L} = -\sum_{\mathbf{v}} \log P(\mathbf{v})
$$
采用对比散度（CD）算法近似梯度，更新规则为：
$$
\Delta W_{ij} = \epsilon \left(\langle v_i h_j \rangle_{\text{data}} - \langle v_i h_j \rangle_{\text{recon}}\right)
$$
其中 $\epsilon$ 为学习率，$\langle \cdot \rangle_{\text{data}}$ 和 $\langle \cdot \rangle_{\text{recon}}$ 分别为数据分布和重构分布的期望。

---
#### **二、整体模块架构与训练模式**
代码实现了基于PyTorch的深度信念网络(DBN)，采用分层架构设计，支持从无监督预训练到有监督学习的完整流程。

- **模块架构**：
  -  **DBNPretrainer**  
      - 实现多层RBM的堆叠与逐层无监督预训练，提供特征提取接口。  
  -  **AbstractSupervisedDBN**  
      - 定义DBN监督学习的通用接口用于支持多模式训练策略，包括预训练、微调、分类器训练与预测等抽象方法。  
  -  **AbstractSupervisedDBNClassifier**  
      - 基于`AbstractSupervisedDBN`，实现PyTorch下的通用工具方法封装，包括特征提取、分类器集成(逻辑回归，支持向量机以及随机森林等)等。
  -  **SupervisedDBNClassification**  
      - 具体分类任务实现、微调网络构建以及训练。

- **训练模式**：
  1. 无监督模式: 仅预训练，用于特征提取
  2. 分类器模式: 预训练 + 下游分类器训练
  3. 微调网络模式: 预训练 + 网络反向传播微调
   
---
#### **三、核心类功能和接口概述**

##### **1. 核心类 `DBNPretrainer（无监督预训练DBN）`**
- **关键参数**：
  - `hidden_layers_structure`：隐层单元数（默认两层[100, 100]）
  - `learning_rate_rbm`：RBM学习率（默认0.1）
  - `n_epochs_rbm`：每层RBM训练轮数（默认10）
  - `batch_size`：批大小（默认100）
  - `verbose`：打印训练信息（默认True）
  - `shuffle`：数据打乱（默认True）
  - `drop_last`：是否丢弃最后不足batch的样本（默认False）
  - `random_state`：随机种子
- **设备支持**：自动选择GPU（`cuda`）或CPU
- **核心方法**
  - **创建RBM层 (`create_rbm_layer` 方法)**
    - **初始化RBM**：使用 `RestrictedBoltzmannMachine` 定义可见层与隐层维度。
  - **单批次训练步骤 (`_train_batch` 方法)**
    1. **正相（Positive Phase）**：计算隐层激活概率 $ P(\mathbf{h}|\mathbf{v}) $。
    2. **负相（Negative Phase）**：通过模拟退火采样器（`SimulatedAnnealingOptimizer`）生成重构样本。
    3. **目标函数**：最小化能量函数加权重衰减（L2正则化）。
    4. **反向传播**：更新权重和偏置。
  - **单层RBM训练 (`_train_rbm_layer` 方法)**
      - **初始化优化器**：采用随机梯度下降（SGD）优化参数。
      - **DataLoader处理批量数据**
  - **预训练堆叠RBM (`fit` 方法)**
  - **特征变换，逐层提取特征 (`transform` 方法)**
##### **2. 核心类 `AbstractSupervisedDBN（抽象接口定义）`**
- **关键参数**：
  - `fine_tuning`：模式选择（默认False）
  - `learning_rate`：微调学习率（默认0.1）
  - `n_iter_backprop`：反向传播迭代次数（默认100）
  - `l2_regularization`：L2正则化（默认1e-4）
  - `activation_function`：激活函数（默认'sigmoid'）
  - `dropout_p`：Dropout概率（默认0.0）
##### **3. 核心类 `AbstractSupervisedDBNClassifier（分类器模式具体实现，以及微调网络构建工具）`**
- **关键参数**：
  - `classifier_type`：支持多种分类器（默认逻辑回归）
  - `clf_C`：正则化强度（默认1.0）
  - `clf_iter`：迭代次数（默认100）
##### **4. 核心类 `SupervisedDBNClassification（具体分类实现）`**
- **微调网络构建**：使用预训练的权重来初始化(默认两层RBM，以及无Dropout层)
  - 网络结构：输入层 → [线性层 + 激活函数 + Dropout] × N → 输出层
  - 线性层：使用对应的RBM的权重初始化
  - 输出层：随机初始化，在微调阶段学习

- **训练策略**：
  - 使用预训练权重初始化
  - CrossEntropyLoss损失函数
  - SGD优化器 + L2正则化
  - 支持Dropout防止过拟合

##### **5. 其他内容**

- **数据加载 (`load_data` 方法)**

    - 数据集：使用 `sklearn.datasets.load_digits`（8x8手写数字图像）。
    - 增强：对原始图像进行上下左右平移，扩展数据集。

- **训练过程可视化 (`_visualize_training_progress` 方法, 设置`plot_img=True`)**

    - 权重与梯度：实时监控权重矩阵及其梯度变化。
    - 生成样本：实时展示模型"生成"新样本的能力
    - 重建样本：可视化重建误差的演变

- **结果可视化 (`RBMVisualizer`类)**

    - 训练后RBM权重可视化
    - 分类任务结果: 混淆矩阵可视化
    - 重建样本：训练完成后对测试图像进行编码-解码得到的重建

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import time
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)

seed = 42
# PyTorch
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)       # 为当前GPU设置
    torch.cuda.manual_seed_all(seed)   # 为所有GPU设置
# Python
random.seed(seed)
# NumPy
np.random.seed(seed)

In [ ]:
from supervised_dbn_digits import load_data, RBMVisualizer, SupervisedDBNClassification

def save_network_structure(dbn, filename_prefix):
    """保存网络结构信息到文件"""
    structure = dbn.get_network_structure()
    
    # 保存为JSON
    import json
    if not os.path.exists("results"):
        os.makedirs("results")
    with open(f"results/{filename_prefix}_structure.json", 'w') as f:
        json.dump(structure, f, indent=2)
    
    # 保存为文本摘要
    with open(f"results/{filename_prefix}_summary.txt", 'w') as f:
        f.write("DBN Model Summary\n")
        f.write("=================\n\n")
        f.write(f"Pre-train layers: {structure['pretrain_layers']}\n")
        f.write(f"Fine-tune layers: {structure['fine_tune_layers']}\n")
        f.write(f"Hidden units: {structure['hidden_units']}\n")
        f.write(f"Number of classes: {structure['num_classes']}\n")
        f.write(f"Training mode: {structure['mode']}\n")
    
    print(f"Network structure saved to results/{filename_prefix}_*")

def demonstrate_both_modes():
    """演示两种模式的使用"""
    # 加载数据
    X_train, X_test, y_train, y_test = load_data(plot_img=False)
    
    # 模式1: 使用微调网络
    print("=== Mode 1: Fine-tuning Network ===")
    dbn_fine_tune = SupervisedDBNClassification(
        hidden_layers_structure=[128, 256],
        learning_rate_rbm=0.1,
        n_epochs_rbm=2,
        batch_size=64,
        fine_tuning=True,  # 启用微调
        learning_rate=0.1,
        n_iter_backprop=100,
        l2_regularization=1e-4,
        activation_function='sigmoid',
        verbose=True,
        random_state=42
    )

    # 训练模型
    dbn_fine_tune.fit(X_train, y_train)

    # 评估模型
    acc = dbn_fine_tune.score(X_test, y_test)*100
    print(f"Fine-tuning network testing accuracy: {acc:.2f}%")

    # 获取训练后的网络结构信息
    structure_info = dbn_fine_tune.get_network_structure()
    print("\n=== Trained Network Structure ===")
    for key, value in structure_info.items():
        print(f"{key}: {value}")

    # 保存网络结构信息
    save_network_structure(dbn_fine_tune, "fine_tune_analysis")
    
    # 模式2: 使用下游分类器
    print("\n=== Mode 2: Pipline Classifier ===")
    dbn_classifier = SupervisedDBNClassification(
        hidden_layers_structure=[128],
        learning_rate_rbm=0.1,
        n_epochs_rbm=2,
        batch_size=64,
        fine_tuning=False,  # 使用分类器
        # 选项: logistic, svm, random_forest
        classifier_type = 'logistic', 
        clf_C=500,
        clf_iter=1000,
        verbose=True,
        random_state=42
    )

    # 训练模型
    dbn_classifier.fit(X_train, y_train)

    # 评估模型
    acc2 = dbn_classifier.score(X_test, y_test)*100
    print(f"Classifier testing accuracy: {acc2:.2f}%")

    # 获取训练后的网络结构信息
    structure_info = dbn_classifier.get_network_structure()
    print("\n=== Trained Network Structure ===")
    for key, value in structure_info.items():
        print(f"{key}: {value}")

    # 保存网络结构信息
    save_network_structure(dbn_classifier, "classifier_analysis") 
    
    # 特征提取示例
    print("\n=== Feature Extraction ===")
    features = dbn_classifier.transform(X_test)
    print(f"Original shape: {X_test.shape}")
    print(f"Feature shape: {features.shape}")
    
    return dbn_fine_tune, dbn_classifier

if __name__ == "__main__":
    model1, model2 = demonstrate_both_modes()
    
    # 保存模型参数
    print("\n=== Save Parameters ===")
    model1.save_parameters(file_prefix="fine_tune")
    model2.save_parameters(file_prefix="classifier")

